In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import pandas as pd
import torch.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from focal_loss.focal_loss import FocalLoss
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv('Aggregated User Data.csv')
data.head()

,UUID,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,...,label:STAIRS_-_GOING_DOWN,label:ELEVATOR,label:OR_standing,label:AT_SCHOOL,label:PHONE_IN_HAND,label:PHONE_IN_BAG,label:PHONE_ON_TABLE,label:WITH_CO-WORKERS,label:WITH_FRIENDS,label_source
0,00EABED2-271D-49D8-B599-1D4A09240601,1444079161,0.996815,0.003529,-0.002786,0.006496,0.995203,0.996825,0.998502,1.748756,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
1,00EABED2-271D-49D8-B599-1D4A09240601,1444079221,0.996864,0.004172,-0.003110,0.007050,0.994957,0.996981,0.998766,1.935573,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
2,00EABED2-271D-49D8-B599-1D4A09240601,1444079281,0.996825,0.003667,0.003094,0.006076,0.994797,0.996614,0.998704,2.031780,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
3,00EABED2-271D-49D8-B599-1D4A09240601,1444079341,0.996874,0.003541,0.000626,0.006059,0.995050,0.996907,0.998690,1.865318,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
4,00EABED2-271D-49D8-B599-1D4A09240601,1444079431,0.997371,0.037653,0.043389,0.102332,0.995548,0.996860,0.998205,0.460806,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2


In [3]:
data = data[(data['UUID'] == '0BFC35E2-4817-4865-BFA7-764742302A2D') | (data['UUID'] == '0A986513-7828-4D53-AA1F-E02D6DF9561B') | (data['UUID'] == '00EABED2-271D-49D8-B599-1D4A09240601')] 
data.drop(columns = ['timestamp'], inplace = True)

# Creating a new dataframe with only sitting, walking, and sleeping data

In [4]:
only_walking = data[(data['label:FIX_walking'] == 1) & (data['label:SITTING'] == 0) & (data['label:SLEEPING'] == 0)]
only_walking = only_walking.iloc[:,1:27]
only_walking['label'] = "WALKING"

only_sitting = data[(data['label:FIX_walking'] == 0) & (data['label:SITTING'] == 1) & (data['label:SLEEPING'] == 0)]
only_sitting = only_sitting.iloc[:,1:27]
only_sitting['label'] = "SITTING"

only_sleeping = data[(data['label:FIX_walking'] == 0) & (data['label:SITTING'] == 0) & (data['label:SLEEPING'] == 1)]
only_sleeping = only_sleeping.iloc[:,1:27]
only_sleeping['label'] = "SLEEPING"

df = pd.concat([only_walking, only_sitting, only_sleeping], axis = 0)
df.head()

,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,raw_acc:magnitude_spectrum:log_energy_band0,...,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,raw_acc:3d:ro_xy,raw_acc:3d:ro_xz,raw_acc:3d:ro_yz,label
155,1.003052,0.139768,0.115192,0.273656,0.991920,0.996238,0.999979,1.328158,6.674634,5.027699,...,-0.051719,-0.168622,-0.866269,0.250667,0.343037,0.252212,-0.030830,0.143926,-0.530033,WALKING
156,1.005091,0.232927,0.190056,0.292741,0.826441,0.948630,1.176819,2.745002,6.658351,5.013784,...,0.255740,-0.433965,-0.801786,0.276395,0.190099,0.235251,0.105712,0.016350,0.027738,WALKING
157,0.988339,0.025706,-0.012189,0.044177,0.979990,0.987836,0.996713,1.783550,6.684273,5.043187,...,-0.028900,-0.594037,-0.772274,0.136729,0.065971,0.065611,-0.603787,0.630796,-0.799983,WALKING
158,1.005461,0.105208,0.128558,0.199233,0.992822,0.994770,0.997684,1.459081,6.679381,5.039396,...,0.152049,-0.164488,-0.791628,0.272913,0.297443,0.426869,-0.372997,0.652161,-0.524372,WALKING
159,1.001464,0.121184,0.129053,0.209801,0.942805,0.996160,1.052143,2.060366,6.677517,5.029112,...,0.149694,-0.014688,-0.913222,0.299310,0.237561,0.122455,0.601753,-0.072091,-0.003273,WALKING


# Interpolating acceleration columns with average values

In [5]:
def interpolation(df):
    col_to_avg = list(df.columns) #Start with keeping all the columns as columns to use an average interpolation on
    for k in range(len(list(df.columns))):
        if list(df.columns)[k].startswith(('discrete', 'label')): #Remove label and discrete columns from col_to_avg
            col_to_avg.remove(list(df.columns)[k])
    
    df_with_avg = df[col_to_avg].fillna(df[col_to_avg].mean()) #Interpolate nan columns for all continuous-valued columns with average
    
    col_to_zero = list(df.columns)
    for k in range(len(list(df.columns))):
        if not list(df.columns)[k].startswith(('discrete', 'label')): #Remove all columns except label and discrete
            col_to_zero.remove(list(df.columns)[k])
    
    df_with_zero = df[col_to_zero].fillna(0) #Interpolate nan values for label and discrete columns with 0
    
    return pd.concat([df_with_avg, df_with_zero], axis = 1)

In [6]:
df.iloc[:,:-1] = interpolation(df.iloc[:,:-1]) #Interpolate acceleration columns
df = df.values
df

array([[1.003052, 0.139768, 0.115192, ..., 0.143926, -0.530033,
        'WALKING'],
       [1.005091, 0.232927, 0.190056, ..., 0.01635, 0.027738, 'WALKING'],
       [0.988339, 0.025706, -0.012189, ..., 0.630796, -0.799983,
        'WALKING'],
       ...,
       [1.000489, 0.001597, 0.000608, ..., -0.131635, 0.020957,
        'SLEEPING'],
       [1.000168, 0.003089, -0.0016879999999999998, ...,
        -0.08845900000000001, -0.090014, 'SLEEPING'],
       [1.019187, 0.143322, 0.24915, ..., 0.19007, -0.548567, 'SLEEPING']],
      dtype=object)

In [7]:
X = df[:, :-1]
y = df[:, -1]

y = LabelEncoder().fit_transform(y)

# SMOTE Oversampling

In [8]:
largest_volume = len(only_sitting)

In [9]:
oversample = SMOTE(sampling_strategy = {0:largest_volume, 1: int(largest_volume * 0.9), 2: int(largest_volume * 0.9)})
X, y = oversample.fit_resample(X, y)

In [10]:
X.shape ### all classes are now 90% the size as the largest class

(9405, 26)

# Train/Test Split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[-3.15517740e-01, -5.38737640e-01, -5.01308721e-01, ...,
         2.99127210e-02, -5.54436904e-01,  1.69975761e-01],
       [ 1.07736584e+00,  2.09961934e+00,  1.75483757e+00, ...,
         1.17659084e+00,  1.78604535e+00,  1.41532197e+00],
       [ 2.05570235e+00,  2.87854416e+00,  2.50773114e+00, ...,
        -1.52324227e+00, -1.85445700e+00,  1.32070940e-03],
       ...,
       [-9.47367666e-02,  3.86790634e-01,  1.08127681e+00, ...,
        -8.85930074e-01,  2.78536609e-01,  9.34868194e-01],
       [-4.77397881e-01, -5.40475481e-01, -5.10907239e-01, ...,
        -1.96484688e-01,  2.65750649e-01,  1.92032179e-01],
       [-3.19514026e-01, -5.35955422e-01, -5.14271595e-01, ...,
        -1.21032628e-01, -6.60322915e-01, -1.56879407e-01]])

# Defining the model

In [12]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = 26):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            classifier_block(10, 5),
            nn.Linear(5, 3)
        )
    def forward(self, x):
        #softmax = nn.Softmax(dim = 1)
        return self.network(x)

# Hyperparameters

In [13]:
model = Classifier()
lr = 0.01
n_epochs = 5000
batch_size = 100

### 2000 epochs 75 batch size 0.001 learning rate
#criterion = FocalLoss(alpha = 0.2, gamma = 5)

### Class 0 --> Sitting, Class 1 --> Sleeping, Class 2 --> Walking 
# weights = torch.tensor([1.0, 2.07, 13], dtype=torch.float32)
# weights = weights / weights.sum()
# normedWeights = torch.FloatTensor(weights)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

# Training Loop

In [14]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        #_, encoded_labels = torch.max(labels, dim = 1) 
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1.0)
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')
#     softmax = nn.Softmax(dim = 1)
#     _, preds = torch.max(softmax(model(features.float())), dim = 1)
#     _, answers = torch.max(labels, dim = 1)
#     print(metrics.confusion_matrix(answers, preds))

Epoch 1, Loss: 61.914939880371094, Final Batch Loss: 0.5787522792816162
Epoch 2, Loss: 43.479945838451385, Final Batch Loss: 0.49985381960868835
Epoch 3, Loss: 37.10497206449509, Final Batch Loss: 0.5697962045669556
Epoch 4, Loss: 34.42878630757332, Final Batch Loss: 0.5309316515922546
Epoch 5, Loss: 33.146532863378525, Final Batch Loss: 0.44782182574272156
Epoch 6, Loss: 31.959238052368164, Final Batch Loss: 0.4773502051830292
Epoch 7, Loss: 31.380652606487274, Final Batch Loss: 0.5476853251457214
Epoch 8, Loss: 29.973882794380188, Final Batch Loss: 0.3323228657245636
Epoch 9, Loss: 29.89193430542946, Final Batch Loss: 0.27622687816619873
Epoch 10, Loss: 28.945295870304108, Final Batch Loss: 0.5681399703025818
Epoch 11, Loss: 28.669544383883476, Final Batch Loss: 0.31730687618255615
Epoch 12, Loss: 29.09804640710354, Final Batch Loss: 0.5204495191574097
Epoch 13, Loss: 28.572676479816437, Final Batch Loss: 0.2724040448665619
Epoch 14, Loss: 28.24705398082733, Final Batch Loss: 0.49994

Epoch 223, Loss: 20.428039133548737, Final Batch Loss: 0.1905151605606079
Epoch 224, Loss: 20.17463594675064, Final Batch Loss: 0.3808218538761139
Epoch 225, Loss: 19.874544888734818, Final Batch Loss: 0.4148314893245697
Epoch 226, Loss: 19.80954022705555, Final Batch Loss: 0.19947309792041779
Epoch 227, Loss: 19.463782109320164, Final Batch Loss: 0.37088537216186523
Epoch 228, Loss: 19.32512402534485, Final Batch Loss: 0.2679889500141144
Epoch 229, Loss: 19.921959213912487, Final Batch Loss: 0.21461504697799683
Epoch 230, Loss: 21.28864711523056, Final Batch Loss: 0.4267631471157074
Epoch 231, Loss: 19.902650356292725, Final Batch Loss: 0.1695658564567566
Epoch 232, Loss: 19.500688284635544, Final Batch Loss: 0.4064972698688507
Epoch 233, Loss: 19.96051025390625, Final Batch Loss: 0.46503880620002747
Epoch 234, Loss: 21.091968908905983, Final Batch Loss: 0.21735990047454834
Epoch 235, Loss: 19.176734909415245, Final Batch Loss: 0.22326652705669403
Epoch 236, Loss: 19.689459905028343, 

Epoch 334, Loss: 19.451763913035393, Final Batch Loss: 0.1586388349533081
Epoch 335, Loss: 20.024198949337006, Final Batch Loss: 0.2893405258655548
Epoch 336, Loss: 20.195164293050766, Final Batch Loss: 0.22538121044635773
Epoch 337, Loss: 19.39754730463028, Final Batch Loss: 0.12582819163799286
Epoch 338, Loss: 19.82411402463913, Final Batch Loss: 0.27887648344039917
Epoch 339, Loss: 18.29597172141075, Final Batch Loss: 0.24395525455474854
Epoch 340, Loss: 19.0078097730875, Final Batch Loss: 0.1417500525712967
Epoch 341, Loss: 19.132145911455154, Final Batch Loss: 0.08185868710279465
Epoch 342, Loss: 18.96375361084938, Final Batch Loss: 0.17884445190429688
Epoch 343, Loss: 19.38954772055149, Final Batch Loss: 0.25385117530822754
Epoch 344, Loss: 23.042623475193977, Final Batch Loss: 0.39491796493530273
Epoch 345, Loss: 21.793663933873177, Final Batch Loss: 0.40281569957733154
Epoch 346, Loss: 21.33381250500679, Final Batch Loss: 0.26134657859802246
Epoch 347, Loss: 22.240606665611267,

Epoch 445, Loss: 18.175763212144375, Final Batch Loss: 0.39457401633262634
Epoch 446, Loss: 18.891305536031723, Final Batch Loss: 0.3136903941631317
Epoch 447, Loss: 18.888969741761684, Final Batch Loss: 0.17925722897052765
Epoch 448, Loss: 18.900128036737442, Final Batch Loss: 0.32870665192604065
Epoch 449, Loss: 19.6266782656312, Final Batch Loss: 0.1154966726899147
Epoch 450, Loss: 17.851027630269527, Final Batch Loss: 0.3031986951828003
Epoch 451, Loss: 17.791819244623184, Final Batch Loss: 0.38033124804496765
Epoch 452, Loss: 19.212255880236626, Final Batch Loss: 0.5002871155738831
Epoch 453, Loss: 18.44178731739521, Final Batch Loss: 0.09021631628274918
Epoch 454, Loss: 17.180785715579987, Final Batch Loss: 0.2869413495063782
Epoch 455, Loss: 19.159277096390724, Final Batch Loss: 0.1561734676361084
Epoch 456, Loss: 19.71819370985031, Final Batch Loss: 0.384944885969162
Epoch 457, Loss: 19.455268129706383, Final Batch Loss: 0.19714508950710297
Epoch 458, Loss: 18.185780242085457, 

Epoch 666, Loss: 17.625978760421276, Final Batch Loss: 0.10662230849266052
Epoch 667, Loss: 17.54434448480606, Final Batch Loss: 0.06599799543619156
Epoch 668, Loss: 18.302592769265175, Final Batch Loss: 0.1624782830476761
Epoch 669, Loss: 18.721229553222656, Final Batch Loss: 0.30721691250801086
Epoch 670, Loss: 17.78343977034092, Final Batch Loss: 0.549433171749115
Epoch 671, Loss: 17.945493772625923, Final Batch Loss: 0.21450714766979218
Epoch 672, Loss: 18.058285169303417, Final Batch Loss: 0.42132696509361267
Epoch 673, Loss: 17.990264624357224, Final Batch Loss: 0.5457276701927185
Epoch 674, Loss: 17.865622267127037, Final Batch Loss: 0.15408867597579956
Epoch 675, Loss: 19.7989906296134, Final Batch Loss: 0.31782785058021545
Epoch 676, Loss: 18.520083531737328, Final Batch Loss: 0.5556191205978394
Epoch 677, Loss: 17.564872533082962, Final Batch Loss: 0.37931856513023376
Epoch 678, Loss: 17.94339118152857, Final Batch Loss: 0.2764488756656647
Epoch 679, Loss: 18.123665496706963,

...

Epoch 9386, Loss: 13.679813757538795, Final Batch Loss: 0.358955055475235
Epoch 9387, Loss: 14.683639243245125, Final Batch Loss: 0.2948457598686218
Epoch 9388, Loss: 14.432757429778576, Final Batch Loss: 0.1892824023962021
Epoch 9389, Loss: 14.094261303544044, Final Batch Loss: 0.2446477860212326
Epoch 9390, Loss: 14.141267821192741, Final Batch Loss: 0.30247119069099426
Epoch 9391, Loss: 13.816279344260693, Final Batch Loss: 0.0832996815443039
Epoch 9392, Loss: 13.419258143752813, Final Batch Loss: 0.05451732873916626
Epoch 9393, Loss: 14.868938133120537, Final Batch Loss: 0.27279505133628845
Epoch 9394, Loss: 14.366198971867561, Final Batch Loss: 0.21822266280651093
Epoch 9395, Loss: 16.74037905782461, Final Batch Loss: 0.2702226936817169
Epoch 9396, Loss: 14.741278313100338, Final Batch Loss: 0.13362349569797516
Epoch 9397, Loss: 13.919714972376823, Final Batch Loss: 0.06568214297294617
Epoch 9398, Loss: 13.302010633051395, Final Batch Loss: 0.18513362109661102
Epoch 9399, Loss: 13

Epoch 9495, Loss: 13.675688464194536, Final Batch Loss: 0.051531482487916946
Epoch 9496, Loss: 14.445497319102287, Final Batch Loss: 0.29155537486076355
Epoch 9497, Loss: 14.169949769973755, Final Batch Loss: 0.13318584859371185
Epoch 9498, Loss: 14.305475942790508, Final Batch Loss: 0.21581752598285675
Epoch 9499, Loss: 13.709028776735067, Final Batch Loss: 0.3267647922039032
Epoch 9500, Loss: 14.397760141640902, Final Batch Loss: 0.8937997817993164
Epoch 9501, Loss: 14.315635442733765, Final Batch Loss: 0.07401194423437119
Epoch 9502, Loss: 14.177321247756481, Final Batch Loss: 0.400916188955307
Epoch 9503, Loss: 16.221792340278625, Final Batch Loss: 0.07783572375774384
Epoch 9504, Loss: 14.424536846578121, Final Batch Loss: 0.21248900890350342
Epoch 9505, Loss: 14.996615760028362, Final Batch Loss: 0.3140347898006439
Epoch 9506, Loss: 15.177392020821571, Final Batch Loss: 0.4683323800563812
Epoch 9507, Loss: 15.204580284655094, Final Batch Loss: 0.016895540058612823
Epoch 9508, Loss

Epoch 9604, Loss: 13.32269462943077, Final Batch Loss: 0.13723228871822357
Epoch 9605, Loss: 15.296227663755417, Final Batch Loss: 0.3636281490325928
Epoch 9606, Loss: 14.043332368135452, Final Batch Loss: 0.15279638767242432
Epoch 9607, Loss: 14.925515837967396, Final Batch Loss: 0.48588237166404724
Epoch 9608, Loss: 14.866723537445068, Final Batch Loss: 0.1035863384604454
Epoch 9609, Loss: 14.177972793579102, Final Batch Loss: 0.19214148819446564
Epoch 9610, Loss: 13.027501583099365, Final Batch Loss: 0.09508433192968369
Epoch 9611, Loss: 14.213802054524422, Final Batch Loss: 0.04557046666741371
Epoch 9612, Loss: 14.320425517857075, Final Batch Loss: 0.12467855960130692
Epoch 9613, Loss: 14.03813423961401, Final Batch Loss: 0.27956393361091614
Epoch 9614, Loss: 14.056485943496227, Final Batch Loss: 0.0581200085580349
Epoch 9615, Loss: 13.761896938085556, Final Batch Loss: 0.12417802959680557
Epoch 9616, Loss: 14.317148342728615, Final Batch Loss: 0.25303980708122253
Epoch 9617, Loss:

Epoch 9713, Loss: 13.978416856378317, Final Batch Loss: 0.21785683929920197
Epoch 9714, Loss: 13.69022462517023, Final Batch Loss: 0.10800886899232864
Epoch 9715, Loss: 16.567161340266466, Final Batch Loss: 0.28946852684020996
Epoch 9716, Loss: 17.16664206236601, Final Batch Loss: 0.09707345813512802
Epoch 9717, Loss: 13.998151816427708, Final Batch Loss: 0.1840105652809143
Epoch 9718, Loss: 14.137161754071712, Final Batch Loss: 0.16500361263751984
Epoch 9719, Loss: 13.877985004335642, Final Batch Loss: 0.0658435970544815
Epoch 9720, Loss: 13.75637811422348, Final Batch Loss: 0.21946103870868683
Epoch 9721, Loss: 14.713783659040928, Final Batch Loss: 0.45771524310112
Epoch 9722, Loss: 14.061256550252438, Final Batch Loss: 0.17686229944229126
Epoch 9723, Loss: 14.070548877120018, Final Batch Loss: 0.07284091413021088
Epoch 9724, Loss: 13.468037329614162, Final Batch Loss: 0.12126147747039795
Epoch 9725, Loss: 14.891691893339157, Final Batch Loss: 0.5131987929344177
Epoch 9726, Loss: 13.

Epoch 9822, Loss: 14.292263358831406, Final Batch Loss: 0.35499152541160583
Epoch 9823, Loss: 14.886074423789978, Final Batch Loss: 0.6717222332954407
Epoch 9824, Loss: 13.1759700961411, Final Batch Loss: 0.04741854593157768
Epoch 9825, Loss: 13.769577708095312, Final Batch Loss: 0.11929642409086227
Epoch 9826, Loss: 13.493906605988741, Final Batch Loss: 0.09840314835309982
Epoch 9827, Loss: 14.734060354530811, Final Batch Loss: 0.1237758994102478
Epoch 9828, Loss: 14.050439864397049, Final Batch Loss: 0.2550106346607208
Epoch 9829, Loss: 14.663880188018084, Final Batch Loss: 0.5677581429481506
Epoch 9830, Loss: 14.579915650188923, Final Batch Loss: 0.07311161607503891
Epoch 9831, Loss: 13.80675833299756, Final Batch Loss: 0.22385340929031372
Epoch 9832, Loss: 13.35851476714015, Final Batch Loss: 0.047800492495298386
Epoch 9833, Loss: 15.086597561836243, Final Batch Loss: 0.3013255298137665
Epoch 9834, Loss: 13.72036261856556, Final Batch Loss: 0.07840928435325623
Epoch 9835, Loss: 14.

# Evaluation on test data

In [15]:
softmax = nn.Softmax(dim = 1)
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    #_, answers = torch.max(labels, dim = 1)
    print(metrics.confusion_matrix(labels, preds))
    print(metrics.classification_report(labels, preds, digits = 3))

[[562  36  61]
 [ 23 599   5]
 [ 20   3 572]]
              precision    recall  f1-score   support

           0      0.929     0.853     0.889       659
           1      0.939     0.955     0.947       627
           2      0.897     0.961     0.928       595

    accuracy                          0.921      1881
   macro avg      0.921     0.923     0.921      1881
weighted avg      0.922     0.921     0.921      1881

